# Selección de Fenotipos para finetuning de BioBERT
Domingo Méndez García. [domingo.mendezg@um.es](mailto:domingo.mendezg@um.es) [github.com/user/DgoMndez](https://github.com/DgoMndez)
* Referencias:
  * Modelo de partida: [pritamdeka](https://huggingface.co/pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb)
  * Ontología: HPO versión https://github.com/obophenotype/human-phenotype-ontology/releases/tag/v2022-12-15

TODO:
* Breve sobre el problema y modelo que estamos tratando.
* Resultados del finetuning anterior: ¿por qué cambiar los fenotipos?
* Análisis de la ontología.
  * IC y profundidad: las variables a tener en cuenta.
  * Método de selección.
  * Resultados.
  * Decisión final informada.
  
## Problema: representación de fenotipos de HPO

El modelo BERT es un Sentence-Transformer que mapea textos a vectores de 768 componentes que llamamos "embeddings", y puede adaptarse a diferentes tareas. Está especializado en textos científicos y médicos pero queremos fine-tunearlo usando un corpus de abstracts de PUBMED para mejorar su desempeño como etapa en PhenoLinker, un sistema que infiere relaciones entre genes y fenotipos para predecir patogenicidad. Los fenotipos que consideramos son los de la subontología Phenotypic Abnormality de HPO (que abreviamos HPO:PA). El objetivo entonces es conseguir que el embedding del nombre de un fenotipo represente mejor al fenotipo como nodo de la ontología. Para medir esto comparamos la similitud de los fenotipos en HPO (Lin) con la similitud coseno entre embeddings.

## Evaluación del experimento de finetuning

Notebook del experimento en [results-lprogress.ipynb](https://github.com/DgoMndez/DL-patogen-colab-DIIC/blob/main/src/fine-tuning/evaluation/results-lprogress.ipynb), con un resumen de cómo se ha entrenado al principio.

* Medidas:

![Pearson correlation vs Batches](./figures/pearson.png)

![Spearman correlation vs Batches](./figures/spearman.png)

![MSE vs Batches](./figures/spearman.png)

* **Conclusiones**: Los mejores scores de evaluación se alcanzan a los pocos batches. Este finetuning no ha funcionado bien porque se alcanza el límite de aprendizaje muy pronto, pero no se percibe sobreajuste porque la tendencia del score train y test se parece.

* **Justificación**:
  * Distintas funciones de pérdida (BatchAllTripletLoss, CosineSimilarityLoss) o distintos hiperparámetros (lr, weight_decay, margin) pueden dar mejores resultados pero no creo que sea el factor determinante.
  * La selección de fenotipos y el tamaño del índice de etiquetas es el factor determinante. Los nodos hoja no están bien representados en la ontología (casi todos tienen similitud lin ~ 0 seguramente porque no aparecen frecuentemente en la BD usada para calcular las similitudes). Esta es la principal explicación de los malos resultados del experimento: estos fenotipos hoja no son útiles para la evaluación y no representan bien HPO. Consecuentemente, hay que volver a obtener un corpus de abstracts con las búsquedas de los nuevos fenotipos y volver a preparar nuevos pares de evaluación y test.

![Lin histogram](./figures/lin.png)

Como vemos la gran mayoría de pares de fenotipos tanto de evaluación como de test tienen similitud 0. La similitud Lin se calcula a partir del IC de cada uno de los fenotipos y del de su ancestro común más profundo.

![IC distribution](./figures/ic-dist-0.png) 

La distribución del IC de los fenotipos es bimodal por la gran cantidad de fenotipos con IC=0 ("nulos"). Estos fenotipos estimamos que causan problemas porque: la medida de similitud no es fiable (no hay ejemplos en la ontología para calcular el IC) y es muy probable que no se encuentren suficientes papers en PUBMED sobre ellos.

"The information content of each node in the HPO can be estimated through its frequency among annotations of the entire OMIM corpus." - [The Human Phenotype Ontology: A Tool for Annotating and Analyzing Human Hereditary Disease](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2668030/)

## Análisis de la ontología

A partir de los resultados anteriores queremos seleccionar un conjunto de fenotipos etiqueta que represente HPO:PA y nos sirva para entrenar, que llamaremos índice. El primer índice de fenotipos era una muestra de tamaño 100 de los nodos hoja de HPO:PA y no funcionó como deseábamos. Para obtener un mejor conjunto de etiquetas (fenotipos) hemos analizado HPO:PA para tomar una decisión.

### IC y profundidad

Tanto el IC como la profundidad son estimadores de la especifidad de un término de HPO. La profundidad se basa únicamente en la estructura de la ontología mientras que el IC se basa en la frecuencia del término (y sus hijos) en el corpus de referencia para la ontología. Por eso usaremos el IC medio de un índice como medida de bondad. Para seleccionar el índice "cortamos" el árbol a cierta profundidad $d$ y nos quedamos con los nodos hoja de ese subárbol:
* Escogemos una profundidad $d$.
* Seleccionamos todos los nodos hoja a profundidad menor que $d$.
* Seleccionamos todos los nodos a profundidad $d$.
* Quitamos todos los nodos nulos (IC=0).
* El índice será una muestra de tamaño 2000 de los fenotipos que quedan. El tamaño viene determinado por la capacidad de búsqueda de abstracts en PUBMED y la potencia de cálculo para el finetuning. Antes habíamos entrenado con un tamaño 100 de índice y una CPU, tardando 8h, ahora hemos tomado un índice 20 veces mayor esperando que con GPU tengamos un finetuning mucho más rápido.

### Resultados del análisis

In [8]:
import pandas as pd
dfDepth = pd.read_csv('results/depth_count.csv', sep='\t')
columnsDrop = ['subtree', 'subtreePercent', 'fullCover', 'sample', 'out']
display(dfDepth.drop(columns=columnsDrop))

,depth,count,leafs,chosen,mean,var,trueCount,zeros,notZeros,meanGTZ,varGTZ,trueMean,trueVar,sampleCover,sampleCoverPercent
0,0,1,0,1,0.000817,NaN,1.0,0.0,1.0,0.000817,NaN,0.000817,NaN,6106,100.000000
1,1,23,0,23,1.177510,0.920597,23.0,0.0,23.0,1.177510,0.920597,1.177510,0.920597,6106,100.000000
2,2,155,28,155,3.186360,5.403676,138.0,17.0,138.0,3.578882,4.659159,3.578882,4.659159,6106,100.000000
3,3,800,318,828,4.368461,8.009591,680.0,138.0,662.0,5.279107,4.867119,5.308145,4.819274,6106,100.000000
4,4,2157,1198,2503,4.306913,9.162556,1846.0,535.0,1622.0,5.727504,4.045136,5.876124,3.957335,6106,100.000000
5,5,3789,2502,5333,4.230784,10.081808,3610.0,1158.0,2631.0,6.092907,3.170809,6.336812,2.997269,3835,62.807075
6,6,3696,2569,7742,4.321174,11.109246,4942.0,1227.0,2469.0,6.468634,2.735594,6.688773,2.387720,2628,43.039633
7,7,2886,1985,9501,3.920541,12.640566,5688.0,1217.0,1669.0,6.779318,2.471154,6.905436,2.055930,2267,37.127416
8,8,1870,1535,10470,2.890284,12.384680,5916.0,1077.0,793.0,6.815676,2.437992,6.978093,1.920782,2092,34.261382
9,9,678,543,10813,3.428010,12.456000,6049.0,330.0,348.0,6.678709,2.528724,7.003977,1.859949,2039,33.393384


**Decisión**: tomar la selección (sin nulos) a profundidad $d=10$ porque:
* El IC medio de la selección crece con la profundidad, pero se estanca a esa profundidad 10 en las centésimas, aparte de que a partir de esa profundidad se añaden muy pocos nodos no nulos.
* Con una muestra de tamaño 2000 de la selección a profundidad 10 cubrimos un 32.82% de los nodos hoja no nulos de la ontología, que es un porcentaje suficiente.
* Importante considerar que si hacemos la muestra completa de 6106 nodos no nulos cubrimos el 100%.
* La clave ha sido quitar los nulos, lo que ha dado unos valores de IC medio y sampleCoverPercent aceptables a partir de $d=5$.